<a href="https://colab.research.google.com/github/guilhermelaviola/BIArchitectureAndBigData/blob/main/Class10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Case Study Context**

This case study focuses on a mid-sized retail company facing challenges with inventory management and sustainability. The goal is to apply inventory optimization and sustainability analysis strategies to improve the company’s operational efficiency.

The company has a variety of products with different life cycles and market demands, making inventory management complex and costly. Through detailed analysis of sales, procurement, and inventory data, the company seeks to minimize waste, optimize inventory levels, and adopt a more sustainable and profitable approach to managing its resources.

**Goals**
- Minimize waste by optimizing inventory levels. Identify the most efficient purchasing strategies based on sales, procurement, and inventory data. Evaluate product sales performance to formulate a sustainable inventory management approach. Data Preprocessing.
- Consolidate all CSV files into a unified master dataset. Check for missing or erroneous entries. Standardize date formats for consistent time series analysis. Inventory Analysis.
- Assess inventory status at the beginning and end of the year using BegInvFINAL12312016.csv and EndInvFINAL12312016.csv. Identify products with the highest and lowest inventory presence. Sales Analysis.
- Examine SalesFINAL12312016.csv to identify top sellers and slow-selling products. Analyze sales trends over time, considering variables such as sales quantity, sales price, and date. Purchasing Analysis.
- Evaluate procurement activities using PurchasesFINAL12312016.csv and InvoicePurchases12312016.csv. Investigate purchase volumes from different suppliers, procurement costs, and supply chain processes. Calculate Optimal Inventory Level.
- Determine the optimal inventory level for each product, leveraging sales, procurement, and inventory data. Propose inventory levels adapted to product sales velocity and supply times. Conclusion: The insights gained from these analyses will provide recommendations for managing inventory more efficiently and sustainably, aiming to reduce costs and prevent excess inventory and waste.

In [1]:
# Importing all the necessary libraries:
import gdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Importing all the necessary datasets:
purchase_prices = pd.read_csv('2017PurchasePricesDec.csv')
beg_inv = pd.read_csv('BegInvFINAL12312016.csv')
end_inv = pd.read_csv('EndInvFINAL12312016.csv')
invoice_purchases = pd.read_csv('InvoicePurchases12312016.csv')
purchases = pd.read_csv('PurchasesFINAL12312016.csv')
sales = pd.read_csv('SalesFINAL12312016.csv')

In [ ]:
# Groupting by Brand and Description and summarizing the inventory at the start of the year:
start_summary = beg_inv.groupby(['Brand', 'Description'])['onHand'].sum().sort_values(ascending=False)

# Groupting by Brand and Description and summarizing the inventory at the end of the year:
end_summary = end_inv.groupby(['Brand', 'Description'])['onHand'].sum().sort_values(ascending=False)

# Identifying the most popular productus at the start and end of the year:
top_5_start = start_summary.head(5)
top_5_end = end_summary.head(5)

# Identifying the least popular productus at the start and end of the year:
bottom_5_start = start_summary.tail(5)
bottom_5_end = end_summary.tail(5)

# Dislaying the results:
print('Top 5 products at the start of the year:\n', top_5_start)
print('\nTop 5 products at the end of the year:\n', top_5_end)
print('\nBottom 5 products at the start of the year:\n', bottom_5_start)
print('\nBottom 5 products at the end of the year:\n', bottom_5_end)

In [ ]:
# Finding the most sold produlcts:
most_sold = sales.groupby(['Brand', 'Description']).agg({'SalesQuantity': 'sum'}).sort_values(by='SalesQuantity', ascending=False).head(10)
print(f'10 most sold products:\n{most_sold}\n')

# Finding the least sold produlcts:
least_sold = sales.groupby(['Brand', 'Description']).agg({'SalesQuantity': 'sum'}).sort_values(by='SalesQuantity', ascending=True).head(10)
print(f'10 least sold products:\n{least_sold}\n')

In [ ]:
purchases['PODate'] = pd.to_datetime(purchases['PODate'], errors='coerce')
purchases['ReceivingDate'] = pd.to_datetime(purchases['ReceivingDate'], errors='coerce')
purchases['SupplyDuration'] = (purchases['ReceivingDate'] - purchases['PODate']).dt.days
average_supply_duration = purchases['SupplyDuration'].mean()
print('Average duration of supply (in days):', average_supply_duration)

In [ ]:
purchases['InvoiceDate'] = pd.to_datetime(purchases['InvoiceDate'])
purchases['PayDate'] = pd.to_datetime(purchases['PayDate'])
purchases['PaymentDuration'] = (purchases['PayDate'] - purchases['InvoiceDate']).dt.days
average_payment_duration = purchases['PaymentDuration'].mean()
print('Average payment duration (in days):', average_payment_duration)

In [ ]:
# Setting the histogram styles:
sns.set_style('whitegrid')

# Histogram for the supply duration:
plt.figure(figsize=(12, 6))
sns.histplot(purchases['SupplyDuration'], kde=True, bins=30, color='coral')
plt.title('Supply Duration')
plt.xlabel('Supply Duration (in days)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Histogram for the payment duration:
plt.figure(figsize=(12, 6))
sns.histplot(purchases['PaymentDuration'], kde=True, bins=30, color='teal')
plt.title('Payment Duration')
plt.xlabel('Payment Duration (in days)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Determining the sales period:
sales['SalesDate'] = pd.to_datetime(sales['SalesDate'])
start_date = sales['SalesDate'].min()
end_date = sales['SalesDate'].max()
total_days = (end_date - start_date).days
total_days

In [ ]:
# Calculating the sale speed for each product:
sales_velocity = sales.groupby(['Brand', 'Description']).agg(Total_Sales=('SalesQuantity', 'sum')).reset_index()
sales_velocity['Sales_Per_Day'] = sales_velocity['Total_Sales'] / total_days
sales_velocity['Sales_Per_Day']

In [ ]:
# Calculating the delivery time:
purchases.loc[:, 'Lead_Time'] = (purchases['ReceivingDate'] - purchases['PODate']).dt.days
lead_times = purchases.groupby(['Brand', 'Description']).agg(Avg_Lead_Time=('Lead_Time', 'mean')).reset_index()
lead_times

In [ ]:
# Merging Sales and Purchase Data:
merged_data = pd.merge(sales_velocity, lead_times, on=['Brand', 'Description'], how='left')
merged_data

In [ ]:
# Calculating Safety Stock using maximum sales for each product:
max_vendas = sales.groupby(['Brand', 'Description']).agg(Max_Vendas_Diarias=('SalesQuantity', 'max')).reset_index()
merged_data = pd.merge(merged_data, max_vendas, on=['Brand', 'Description'], how='left')
merged_data

In [ ]:
# Calculating the Optimal Stock Level:
merged_data['Optimal_Stock_Level'] = merged_data['Sales_Per_Day'] * merged_data['Avg_Lead_Time']
merged_data['Optimal_Stock_Level']

In [ ]:
# Calculating Safety Stock:
max_sales = sales.groupby(['Brand', 'Description']).agg(Max_Daily_Sales=('SalesQuantity', 'max')).reset_index()
merged_data = pd.merge(merged_data, max_sales, on=['Brand', 'Description'], how='left')
merged_data['Safety_Stock'] = merged_data['Max_Daily_Sales'] - merged_data['Sales_Per_Day']
merged_data['Recommended_Stock_Level'] = merged_data['Optimal_Stock_Level'] + merged_data['Safety_Stock']
merged_data['Recommended_Stock_Level']

In [ ]:
# Sorting data by Recommended Stock Level for better visualization:
ordered_data = merged_data.sort_values(by='Recommended_Stock_Level', ascending=False)

# Plotando
plt.figure(figsize=(15, 10))
sns.barplot(x='Recommended_Stock_Level', y='Description', data=ordered_data.head(20), palette='viridis')  # mostrando os 20 produtos principais para melhor visualização
plt.xlabel('Recommended Stock Level')
plt.ylabel('Product Description')
plt.title('Recommended Stock Levels for Top 20 Products')
plt.tight_layout()
plt.show()